In [3]:


from Gate import Transformer2
from util import (get_tensors, get_mask, init_weights, get_std_opt)
from util import (long_loss, surv_loss)
model = Transformer2(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

In [1]:
from Gate import Transformer2
import torch
import numpy as np
from util import (get_tensors, get_mask, init_weights, get_std_opt)
import matplotlib.pyplot as plt
from util import (long_loss, surv_loss)
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Load the pickle file back into a DataFrame
data = pd.read_pickle('data/DIVAT_sim_1000_visit_1000_long_3_seed_123.pkl')
I = data['id'].nunique()

        ## split train/test
random_id = range(I) #np.random.permutation(range(I))
train_id = random_id[0:int(0.7*I)]
test_id = random_id[int(0.7*I):I]

train_data = data[data["id"].isin(train_id)]
test_data = data[data["id"].isin(test_id)]

print(train_data.shape)

batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(train_data)
minmax_scaler = MinMaxScaler(feature_range=(-1,1))
train_data.loc[:,["Y1","Y2","Y3"]] = minmax_scaler.fit_transform(train_data.loc[:,["Y1","Y2","Y3"]])
test_data.loc[:,["Y1","Y2","Y3"]] = minmax_scaler.transform(test_data.loc[:,["Y1","Y2","Y3"]])
torch.manual_seed(0)

(8725, 22)


/home/shijimao/Proj1/util.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"id_new"] = df.groupby(by="id").ngroup()


In [8]:
batch_mask.shape

torch.Size([700, 11])

In [ ]:
from Gate import Transformer1
from util import (get_tensors, get_mask, init_weights, get_std_opt)
model = Transformer1(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

loss_values

In [ ]:
import torch
import numpy as np

from Gate import Transformer3
from util import (get_tensors, get_mask, init_weights, get_std_opt)
model = Transformer3(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

loss_values

In [4]:
from metrics import (AUC, Brier, MSE)
n_sim = 0
long_mse = np.zeros((1, 3))  

test_long, test_base, test_mask, e_test, t_test, obs_time = get_tensors(test_data.copy())    
base_0 = test_base[:,0,:].unsqueeze(1)
long_pred = torch.zeros(test_long.shape[0],0,test_long.shape[2]).to('cuda')

model = model.eval()

for j in range(1,test_long.shape[1]):
    dec_long = test_long[:,:j,:]
    print(dec_long.shape)
    dec_base = base_0.expand([-1,dec_long.shape[1],-1])
    print(dec_base.shape)
    out = model.decoder(dec_long.to('cuda'), dec_base.to('cuda'), get_mask(test_mask[:,:j]).to('cuda'), obs_time[:,:j].to('cuda'))
    #long_out,surv_out = model.mmoe_layer(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
    out = model.decoder_pred(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
    long_out = model.long(out)
    long_pred = torch.cat((long_pred, long_out), dim=1)

long_pred = long_pred.detach().cpu().numpy()
long_obs = test_long[:,1:,:].cpu().numpy()
long_mask = test_mask[:,1:].unsqueeze(2).repeat((1,1,long_pred.shape[-1])).cpu().numpy()

long_obs = np.ma.array(long_obs, mask=1-long_mask)
long_obs = long_obs.filled(fill_value=np.nan)

long_mse[0,:] = MSE(long_pred, long_obs)

torch.Size([300, 1, 3])
torch.Size([300, 1, 2])


AttributeError: 'Transformer2' object has no attribute 'decoder_pred'

In [5]:
from metrics import (AUC, Brier, MSE)
n_sim = 0
long_mse = np.zeros((1, 3))  

test_long, test_base, test_mask, e_test, t_test, obs_time = get_tensors(test_data.copy())    
base_0 = test_base[:,0,:].unsqueeze(1)
long_pred = torch.zeros(test_long.shape[0],0,test_long.shape[2]).to('cuda')

model = model.eval()

for j in range(1,test_long.shape[1]):
    dec_long = test_long[:,:j,:]
    dec_base = base_0.expand([-1,dec_long.shape[1],-1])
    out = model.decoder(dec_long.to('cuda'), dec_base.to('cuda'), get_mask(test_mask[:,:j]).to('cuda'), obs_time[:,:j].to('cuda'))
    long_out,surv_out = model.mmoe_layer(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
    long_pred = torch.cat((long_pred, long_out), dim=1)

long_pred = long_pred.detach().cpu().numpy()
long_obs = test_long[:,1:,:].cpu().numpy()
long_mask = test_mask[:,1:].unsqueeze(2).repeat((1,1,long_pred.shape[-1])).cpu().numpy()

long_obs = np.ma.array(long_obs, mask=1-long_mask)
long_obs = long_obs.filled(fill_value=np.nan)

long_mse[0,:] = MSE(long_pred, long_obs)

In [6]:

from Gate import Transformer2

model = Transformer2(d_long=3, d_base=2, d_model=32, nhead=4,
                num_decoder_layers=7)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)
train_long, train_base, train_mask, e_train, t_train, train_obs_time = get_tensors(train_data.copy())
n_epoch = n_epoch 
batch_size = batch_size 
n_sim = 2
obstime = [0,1,2,3,4,5,6,7,8,9,10]
landmark_times = [1,2,3,4,5]
pred_windows = [1,2,3] 
    
loss_values = []

AUC_array = np.zeros((n_sim, len(landmark_times), len(pred_windows)))
iAUC_array = np.zeros((n_sim, len(landmark_times)))
true_AUC_array = np.zeros((n_sim, len(landmark_times), len(pred_windows)))
true_iAUC_array = np.zeros((n_sim, len(landmark_times)))

BS_array = np.zeros((n_sim, len(landmark_times), len(pred_windows)))
iBS_array = np.zeros((n_sim, len(landmark_times)))
true_BS_array = np.zeros((n_sim, len(landmark_times), len(pred_windows)))
true_iBS_array = np.zeros((n_sim, len(landmark_times)))

long_mse = np.zeros((n_sim, 3)) 

for i_sim in range(n_sim):
    if i_sim % 10 == 0:
        print(i_sim)

    np.random.seed(i_sim)  
    ### 
    ### Train 
    for epoch in range(n_epoch):
        running_loss = 0
        train_id = np.random.permutation(train_id)
        for batch in range(0, len(train_id), batch_size):
            optimizer.zero_grad()
                
            indices = train_id[batch:batch+batch_size]
            batch_data = train_data[train_data["id"].isin(indices)]
                
            batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
            batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
            batch_base = batch_base[:,:-1,:].to('cuda')
            batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
            batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
            obs_time = obs_time.to('cuda')
            yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                            obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
            
            loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
            loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
            
            loss = loss1 + loss2
            
            loss.backward()
            scheduler.step()
            running_loss += loss
        loss_values.append(running_loss.tolist())


    ### 
    ### Test 

    ## Survival Prediction using Landmarking

    for LT_index, LT in enumerate(landmark_times):
        
        pred_times = [x+LT for x in pred_windows]
        pred_times = pred_times
        # Only keep subjects with survival time > landmark time
        tmp_data = test_data.loc[test_data["time"]>LT,:]
        tmp_id = np.unique(tmp_data["id"].values)
        tmp_all = data.loc[data["id"].isin(tmp_id),:]
        
        # Only keep longitudinal observations <= landmark time
        tmp_data = tmp_data.loc[tmp_data["obstime"]<=LT,:]

        true_prob_tmp = tmp_all.loc[tmp_all["predtime"].isin(pred_times), ["true"]].values.reshape(-1,len(pred_times))
        true_prob_LT = tmp_all.loc[tmp_all["predtime"]==LT, ["true"]].values
        true_prob_tmp = true_prob_tmp / true_prob_LT # true conditional survival
                
        tmp_long, tmp_base, tmp_mask, e_tmp, t_tmp, obs_time = get_tensors(tmp_data.copy())
        tmp_long = tmp_long.to('cuda')
        tmp_base = tmp_base.to('cuda')
        tmp_mask = tmp_mask.to('cuda')
        e_tmp = e_tmp
        t_tmp = t_tmp
        obs_time = obs_time.to('cuda')
        base_0 = tmp_base[:,0,:].unsqueeze(1)        
        long_0 = tmp_long
        mask_T = torch.ones((long_0.shape[0],1), dtype=torch.bool).to('cuda')
        
        dec_long = long_0
        dec_base = base_0
        
        long_pred = torch.zeros(long_0.shape[0],0,long_0.shape[2]).to('cuda')
        surv_pred = torch.zeros(long_0.shape[0],0,1).to('cuda')
        
        model = model.eval()
        
        for pt in pred_times:
            dec_base = base_0.expand([-1,dec_long.shape[1],-1])
            
            out = model.decoder(dec_long, dec_base, get_mask(tmp_mask), obs_time)
            
            long_out,surv_out = model.mmoe_layer(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
            long_pred = torch.cat((long_pred, long_out), dim=1)
            
            dec_long = torch.cat((dec_long, long_out), dim=1)
            tmp_mask = torch.cat((tmp_mask, mask_T), dim=1)
            obs_time = torch.cat((obs_time, torch.tensor(pt).expand([obs_time.shape[0],1]).to('cuda')),dim=1)
        
        long_pred = long_pred.detach().cpu().numpy()
        surv_pred = surv_pred.squeeze().detach().cpu().numpy()
        surv_pred = surv_pred.cumprod(axis=1)

        auc, iauc = AUC(surv_pred, e_tmp.numpy(), t_tmp.numpy(), np.array(pred_times))
        AUC_array[i_sim, LT_index, :] = auc
        iAUC_array[i_sim, LT_index] = iauc
        auc, iauc = AUC(true_prob_tmp, np.array(e_tmp), np.array(t_tmp), np.array(pred_times))
        true_AUC_array[i_sim, LT_index, :] = auc
        true_iAUC_array[i_sim, LT_index] = iauc
        
        bs, ibs = Brier(surv_pred, e_tmp.numpy(), t_tmp.numpy(),
                            e_train.numpy(), t_train.numpy(), LT, np.array(pred_windows))
        BS_array[i_sim, LT_index, :] = bs
        iBS_array[i_sim, LT_index] = ibs
        bs, ibs = Brier(true_prob_tmp, e_tmp.numpy(), t_tmp.numpy(),
                            e_train.numpy(), t_train.numpy(), LT, np.array(pred_windows))
        true_BS_array[i_sim, LT_index, :] = bs
        true_iBS_array[i_sim, LT_index] = ibs
        
    
    ## Longitudinal Prediction for observed values
    test_long, test_base, test_mask, e_test, t_test, obs_time = get_tensors(test_data.copy())    
    base_0 = test_base[:,0,:].unsqueeze(1)
    long_pred = torch.zeros(test_long.shape[0],0,test_long.shape[2]).to('cuda')
    
    model = model.eval()
    
    for j in range(1,test_long.shape[1]):
        dec_long = test_long[:,:j,:]
        dec_base = base_0.expand([-1,dec_long.shape[1],-1])
    
        out = model.decoder(dec_long.to('cuda'), dec_base.to('cuda'), get_mask(test_mask[:,:j]).to('cuda'), obs_time[:,:j].to('cuda'))
        long_out,surv_out = model.mmoe_layer(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
        long_pred = torch.cat((long_pred, long_out), dim=1)

    
    long_pred = long_pred.detach().cpu().numpy()
    long_obs = test_long[:,1:,:].cpu().numpy()
    long_mask = test_mask[:,1:].unsqueeze(2).repeat((1,1,long_pred.shape[-1])).cpu().numpy()
    
    long_obs = np.ma.array(long_obs, mask=1-long_mask)
    long_obs = long_obs.fillMyModeled(fill_value=np.nan)
    
    long_mse[i_sim,:] = MSE(long_pred, long_obs)

0


KeyError: 'predtime'

In [20]:
for LT_index, LT in enumerate(landmark_times):
        
        pred_times = [x+LT for x in pred_windows]
        pred_times = pred_times
        # Only keep subjects with survival time > landmark time
        tmp_data = test_data.loc[test_data["time"]>LT,:]
        tmp_id = np.unique(tmp_data["id"].values)
        tmp_all = data.loc[data["id"].isin(tmp_id),:]
        
        # Only keep longitudinal observations <= landmark time
        tmp_data = tmp_data.loc[tmp_data["obstime"]<=LT,:]
        
        true_prob_tmp = tmp_all.loc[tmp_all["predtime"].isin(pred_times), ["true"]].values.reshape(-1,len(pred_times))
        true_prob_LT = tmp_all.loc[tmp_all["predtime"]==LT, ["true"]].values
        true_prob_tmp = true_prob_tmp / true_prob_LT # true conditional survival
                
        tmp_long, tmp_base, tmp_mask, e_tmp, t_tmp, obs_time = get_tensors(tmp_data.copy())
        tmp_long = tmp_long.to('cuda')
        tmp_base = tmp_base.to('cuda')
        tmp_mask = tmp_mask.to('cuda')
        e_tmp = e_tmp
        t_tmp = t_tmp
        obs_time = obs_time.to('cuda')
        base_0 = tmp_base[:,0,:].unsqueeze(1)        
        long_0 = tmp_long
        mask_T = torch.ones((long_0.shape[0],1), dtype=torch.bool).to('cuda')
        
        dec_long = long_0
        dec_base = base_0
        
        long_pred = torch.zeros(long_0.shape[0],0,long_0.shape[2]).to('cuda')
        surv_pred = torch.zeros(long_0.shape[0],0,1).to('cuda')
        
        model = model.eval()
        
        for pt in pred_times:
            dec_base = base_0.expand([-1,dec_long.shape[1],-1])
            
            out = model.decoder(dec_long, dec_base, get_mask(tmp_mask), obs_time)
            long_out,surv_out = model.mmoe_layer(out[:,-1,:].unsqueeze(1).to('cuda'), out, test_mask[:,:j].unsqueeze(1).to('cuda'), obs_time[:,j].unsqueeze(1).to('cuda'))
            long_pred = torch.cat((long_pred, long_out), dim=1)
            
            dec_long = torch.cat((dec_long, long_out), dim=1)
            tmp_mask = torch.cat((tmp_mask, mask_T), dim=1)
            obs_time = torch.cat((obs_time, torch.tensor(pt).expand([obs_time.shape[0],1]).to('cuda')),dim=1)
        
        long_pred = long_pred.detach().cpu().numpy()
        surv_pred = surv_pred.squeeze().detach().cpu().numpy()
        surv_pred = surv_pred.cumprod(axis=1)

        auc, iauc = AUC(surv_pred, e_tmp.numpy(), t_tmp.numpy(), np.array(pred_times))
        AUC_array[i_sim, LT_index, :] = auc
        iAUC_array[i_sim, LT_index] = iauc
        auc, iauc = AUC(true_prob_tmp, np.array(e_tmp), np.array(t_tmp), np.array(pred_times))
        true_AUC_array[i_sim, LT_index, :] = auc
        true_iAUC_array[i_sim, LT_index] = iauc
        
        bs, ibs = Brier(surv_pred, e_tmp.numpy(), t_tmp.numpy(),
                            e_train.numpy(), t_train.numpy(), LT, np.array(pred_windows))
        BS_array[i_sim, LT_index, :] = bs
        iBS_array[i_sim, LT_index] = ibs
        bs, ibs = Brier(true_prob_tmp, e_tmp.numpy(), t_tmp.numpy(),
                            e_train.numpy(), t_train.numpy(), LT, np.array(pred_windows))
        true_BS_array[i_sim, LT_index, :] = bs
        true_iBS_array[i_sim, LT_index] = ibs

ValueError: cannot reshape array of size 794 into shape (3)